In [96]:
import pandas as pd
import os

import torch
from torch import nn
import torch.optim as optim
from torch.optim import Adam
from sklearn.model_selection import train_test_split

In [23]:
# change working directory 
os.chdir('C:/Users/ehyun/Documents/Kaggle/NeurIPS 2024/Data')

In [25]:
# Loading in parquet file as dataframe
df = pd.read_parquet('features_withoutdupes.parquet', engine='pyarrow')

In [40]:
df.head()

,fp_0,fp_1,fp_2,fp_3,fp_4,fp_5,fp_6,fp_7,fp_8,fp_9,...,fp_162,fp_163,fp_164,fp_165,MolWt,MolLogP,Unnamed: 0,binds_BRD4,binds_HSA,binds_sEH
0,0,0,0,0,0,1,0,0,0,0,...,1,1,1,0,573.411,1.50200,3574,1,0,0
2,0,0,0,0,0,1,0,0,0,0,...,1,1,1,0,637.045,3.21830,4715,1,0,0
3,0,0,0,0,0,1,0,0,0,0,...,1,1,1,0,621.072,2.10180,7581,1,0,0
4,0,0,0,0,0,1,0,0,0,0,...,1,1,1,0,664.162,2.53100,7605,1,0,0
5,0,0,0,0,0,1,0,0,0,0,...,1,1,1,0,667.118,2.62782,7613,1,0,0


In [33]:
# getting list of column names to create X and Y variables 
columns = list(df.columns)
features = df[columns[:-4]]
outputs = df[columns[-3:]] # ignore the index column

In [70]:
# Get input data (features) as a tensor
X = torch.tensor(features.values)
# Get output data (outputs) as a tensor
y = torch.tensor(outputs.values)

In [71]:
# print first item in each tensor to see if conversion worked
print(X.data[0])
print(y.data[0])

tensor([  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   1.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   1.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   1.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   1.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   1.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   1.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   1.0000,   0.0000,   0.0000,   0.0000,   1.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   1.0000,   1.0000,
          0.0000,   0.0000,   1.0000,   1.0000,   1.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   1.0000,   0.0000,   0.00

In [72]:
# Split the dataset into training (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2,
                                                    random_state = 55) # keep the same random splitting each time

In [111]:
# converting the tensors so that they contain float values
X_train = X_train.float()
y_train = y_train.float()
X_test = X_test.float()
y_test = y_test.float()


print(X_train.size())
print(y_train.size())
print(X_test.size())
print(y_test.size())

torch.Size([2024127, 168])
torch.Size([2024127, 3])
torch.Size([506032, 168])
torch.Size([506032, 3])


In [85]:
# seeing the total number of hits for each protein in train / test set
train_sums = torch.sum(y_train, dim = 0)
test_sums = torch.sum(y_test, dim = 0)
print(train_sums)
print(test_sums)
# BRD4, HSA, sEH

tensor([292560, 281858, 281350])
tensor([73309, 70635, 70883])


In [122]:
# Creating the feedforward neural network

# Defining the class of the model
class DrugTargets(nn.Module): # reference nn.Module from 'nn' import
    def __init__(self):
        super(DrugTargets, self).__init__() # super gives access to methods from the nn.Module class

        # Create the first layer, taking in all inputs (168)
        self.fc1 = nn.Linear(168, 300) # fc = fully connected, 168 = inputs, 300 = outputs
        self.relu = nn.ReLU() # ReLU activation function applied to the inputs
        
        # Create the hidden layers
        self.fc2 = nn.Linear(300, 150) # Hidden layer 1 
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(150, 3) # Hidden layer 2, final output must be for 3 targets
        self.sigmoid = nn.Sigmoid() # Final activation function is sigmoid to output probability for each drug target
    
    def forward(self, x): # Tells the model how to produce the output tensor when an input tensor is given
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

model = DrugTargets()

# Define the loss function and optimizer
loss_fn = nn.BCELoss() # binary cross entropy
opt = optim.Adam(model.parameters(), lr = 0.0001) # lr = learning rate

In [136]:
# Training loop to give batches of data to the model and set number of epochs (rounds of training)

####### CANCELLED THIS CODE BECAUSE IT DIDNT RUN IN TIME ########### 
####### TOO LOW OF BATCH SIZE? ##########
n_epochs = 10
batch_size = 64
 
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X_train[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y_train[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        opt.zero_grad()
        loss.backward()
        opt.step()
    print(f'Finished epoch {epoch}, latest loss {loss.item()}')

Finished epoch 0, latest loss nan


KeyboardInterrupt: 

In [125]:
#### OUTPUTS FROM HERE ARE WITH THE MODEL BEING TRAINED WITH 10 EPOCHS, 1,000,000 BATCH SIZE ####
#### BAD RESULTS#####

# predicting the test set 
y_preds = model(X_test)

In [128]:
accuracy = (y_preds.round() == y_test).float().mean()
# The round() function rounds off the floating point to the nearest integer. 
# The == operator compares and returns a Boolean tensor, which can be converted to floating point numbers 1.0 and 0.0. 
# The mean() function will provide you the count of the number of 1’s 
# (i.e., prediction matches the label) divided by the total number of samples.
print(accuracy)

tensor(0.8585)


In [130]:
y_preds.data[200]

tensor([0.0916, 0.0619, 0.1084])

In [131]:
# Threshold the predictions
threshold = 0.5
y_preds_binary = (y_preds >= threshold).float()

# Calculate True Positives (TP) and False Positives (FP)
TP = torch.sum((y_preds_binary == 1) & (y_test == 1), dim=0)
FP = torch.sum((y_preds_binary == 1) & (y_test == 0), dim=0)

# Compute precision for each target
precision = TP / (TP + FP + 1e-10)  # Adding a small epsilon to avoid division by zero

# Print precision for each target
for i, prec in enumerate(precision):
    print(f'Precision for target {i+1}: {prec.item()}')

Precision for target 1: 0.0
Precision for target 2: 0.0
Precision for target 3: 0.0


In [133]:
num_ones_per_target = torch.sum(y_preds_binary, dim=0)

# Print the number of 1 predictions for each target
for i, num_ones in enumerate(num_ones_per_target):
    print(f'Number of 1 predictions for target {i+1}: {num_ones.item()}')

Number of 1 predictions for target 1: 0.0
Number of 1 predictions for target 2: 0.0
Number of 1 predictions for target 3: 0.0
